![](img/575_banner.png)

# Lecture 7: Self-attention and transformers

UBC Master of Data Science program, 2021-22

Instructor: Varada Kolhatkar

> [Attention is all you need!](https://arxiv.org/pdf/1706.03762.pdf)

## Lecture plan, imports, LO

### Lecture plan 

- Self-attention layers
- Transformer blocks 
- Break
- iClicker questions
- Multihead attention
- Transfer learning 
- Course conclusion

<br><br>

## Imports

In [1]:
import sys
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

pd.set_option("display.max_colwidth", 0)

<br><br>

### Learning outcomes

From this lecture you will be able to 

- Broadly explain the limitations of LSTMs. 
- Explain the idea of self-attention. 
- Describe the three core operations in self-attention. 
- Explain the query, key, and value roles in self-attention. 
- Explain the role of linear projections for query, key, and value in self-attention. 
- Explain transformer blocks. 
- Explain the advantages of using transformers over LSTMs. 
- Broadly explain the idea of multihead attention. 
- Broadly explain the idea of transfer learning. 

<br><br>

### Attributions

This material is heavily based on [Jurafsky and Martin, Chapter 9]((https://web.stanford.edu/~jurafsky/slp3/9.pdf)).

<br><br><br><br>

### Motivation

What kind of neural network models are at the core of all state-of-the-art NLP models (e.g., BERT, GPT2, GPT3)? 

<br><br><br><br>

## Self-attention networks: Transformers

### Limitations of LSTMs

- LSTMs are better than vanilla RNNs in mitigating the problem of loss of distant information caused by vanishing gradients. 
- But the underlying problem remains.
- For long sequences, there is still a loss of relevant information and difficulties in training. 
- Also, inherently sequential nature of LSTMs make them hard to parallelize. So they are slow to train. 
- This led to development of **transformers**. 
    - They are much faster to train compared to LSTMs and much better at capturing long distance dependencies. 
    - They are at the core of all state-of-the-art NLP models (e.g., BERT, GPT2, GPT3). 

### Transformers 

- Transformers provide an approach to sequence processing but they eliminate recurrent connections in RNNs and LSTMs.   
- Similar to RNNs or LSTMs, they map sequences of input vectors $(x_1, \dots, x_n)$ to sequences of output vectors $(y_1, \dots, y_n)$ of the same length.  
- They are made up of **transformer blocks** which are multilayer networks made by combining 
    - simple linear layers,
    - feedforward layers, and 
    - **self-attention layers**, which is the key innovation of transformers. 
- You will see these transformer blocks in modern language model architectures. 
- Let's first focus on self-attention layer. 
- Later we'll see how it fits in the larger transformer blocks. 

![](img/gpt3-transformer-blocks.gif)

[Source](https://jalammar.github.io/how-gpt3-works-visualizations-animations/)

### Self-attention layer

- Self-attention allows a network to directly extract and use information from arbitrarily large contexts without the need to pass it through intermediate recurrent connections as in RNNs. 
- Below is a single backward looking self-attention layer which maps sequences of input vectors $(x_1, \dots, x_n)$ to sequences of output vectors $(y_1, \dots, y_n)$ of the same length. 
- When processing an item at time $t$, the model has access to all of the inputs up to and including the one under consideration. 
- It does not have access to the input beyond the current one. 
- Note that unlike RNNs or LSTMs, each computation can be done independently; it does not depend upon the previous computation which allows us to easily parallelize forward pass and the training of such models. 

![](img/self_attention.png)
<!-- <img src="img/self_attention.png" width="600" height="600"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

### Self-attention: Core idea 
- We want to be able to compare a token of our interest to a collection of other tokens in a way that reveals their relevance in the current context.
- For each token in the sequence, we assign a **weight** based on how relevant it is to the token under consideration. 
- Calculate the output for the current token based on these weights. 

### Example: Calculating the output $y$ for the token _nuts_ in the given context

![](img/self_attention_ex_nuts.png)

<!-- <img src="img/self_attention_ex_nuts.png" width="600" height="600"> -->

### The key operations in self-attention

In order to calculate the output $y_i$

- We score token $x_i$ with all previous tokens $x_j$ by taking the dot product between them. 
$$\text{score}(x_i, x_j) = x_i \cdot x_j$$

- We apply $\text{softmax}$ on these scores to get probability distribution over these scores. 
$$\alpha_{ij} = \text{softmax}(\text{score}(x_i \cdot x_j)), \forall j \leq i$$

- The output is the weighted sum of the inputs seen so far, where the weights correspond to the $\alpha$ values calculated above. 
 $$y_i = \sum_{j \leq i} \alpha_{ij}x_j$$
 
These three operations represent the core of an attention-based approach. These operations can be carried out independently for each input allowing easy parallelism. 

### Query, Key, and Value roles

Note that in the process of calculating outputs corresponding to each input, each input embedding plays three kinds of roles. 

- **Query**: _the current focus of attention_ when being compared to all previous inputs. 
- **Key**: _a preceding input_ being compared to the current focus of attention.    
- **Value**: used to compute the output for the current focus of attention. 

For these three roles transformer introduces three weight matrices: $W^Q, W^K, W^V$. These weights will be used to project each input vector $x_i$ into its role as a key, query, or value.

$$q_i = W^Qx_i$$
$$k_i = W^Kx_i$$
$$v_i = W^Vx_i$$

For now let's assume that all these weight matrices have the same dimensionality and so the projected vectors in each case are going to be of the same size. 

With these projections our equations become: 

- We score the $x_i$ with all previous tokens $x_j$ by taking the dot product between $x_i$'s query vector $q_i$ and $x_j$'s key vector $k_j$:  
$$\text{score}(x_i, x_j) = q_i \cdot k_j$$

- The softmax calculation remains the same but the output calculation for $y_i$ is now based on a weighted sum over the projected vectors $v$:
 $$y_i = \sum_{j \leq i} \alpha_{ij}v_j$$
 

### Self-attention: Calculating the value of $y_3$

![](img/self_attention_ex.png)

<!-- <img src="img/self_attention_ex.png" width="400" height="400"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

### Self-attention example: Calculating the value of $y_3$

- Let's calculate the output of _**nuts**_ in the following sequence with $K, Q, V$ matrices.  
> had bowl nuts 
- Suppose input embedding is of size 300. 
- Suppose the projection matrices $W^k, W^q, W^v$ are of shape $300 \times 100$. 
- So word$_k$, word$_q$, word$_v$ provide 100-dimensional projections of each word corresponding to the key, query and value roles. For example, nuts$_k$, nuts$_q$, nuts$_v$ represent 100-dimensional projections of the word **nuts** corresponding to its key, query, and value roles, respectively.
- The dot products will be calculated between the appropriate query and key projections. In this example, we will calculate the following dot products:
    - $\text{nuts}_q \cdot \text{had}_k$
    - $\text{nuts}_q \cdot \text{bowl}_k$    
    - $\text{nuts}_q \cdot \text{nuts}_k$
- We apply softmax on these dot products. Suppose the softmax output in this toy example is 
\begin{bmatrix} 0.005 & 0.085 & 0.91 \end{bmatrix}
- So we have weights associated with three input words: _had_ (0.005), _bowl_ (0.085) and _nuts_ (0.91)
- We can calculate the output as the weighted sum of the inputs. Here we will use the value projections of the inputs: $0.005 \times \text{had}_v + 0.085 \times \text{bowl}_v + 0.91 \times \text{nuts}_v$
- Since we will be adding 100 dimensional vectors (size of our projections), the dimensionality of the output $y_3$ is going to be 100. 

![](img/self_attention_nuts.png)

<!-- <img src="img/self_attention_ex.png" width="400" height="400"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

### Break

![](img/eva-coffee.png)

### Scaling the dot products

- The result of a dot product can be arbitrarily large and exponentiating such values can lead to numerical issues and problems during training. 
- So the dot products are usually scaled before applying the softmax. 
- The most common scaling is where we divide the dot product by the square root of the dimensionality of the query and the key vectors. 
$$\text{score}(x_i, x_j) = \frac{q_i \cdot k_j}{\sqrt{d_k}}$$


- This is how we calculate a single output of a single time step $i$. 
- Would the output calculation at different time steps be dependent upon each other? 

### Efficient calculations with matrix multiplication 

- $X_{N \times d} \rightarrow$ matrix of all tokens in a sequence of length $N$ with each token represented with a $d$ dimensional embedding. Each row of $X$ is embedding representation of one token of the input. The we can calculate $Q, K, V$ as follows.

$$Q = XW^Q$$
$$K = XW^K$$
$$V = XW^V$$

- With these, we can now calculate all the query-key scores simultaneously as $Q \times K$. 

![](img/self_attention_calc_all.png)

<!-- <img src="img/self_attention_calc_all.png" width="300" height="300"> -->

- We can them apply softmax on all rows and multiply the resulting matrix by $V$.

$$SelfAttention(Q, K, V) = \text{softmax}(\frac{QK}{\sqrt{d_k}})V$$

- Finally, we get output sequence $y_1, \dots, y_n$.   


- What's the problem with the approach above?
    - This process goes a bit too far since the calculation of the comparisons in $QK$ results in a score for each value to each key value, _including those that follow the query_. 
    - Is this appropriate in the setting of language modeling? 

![](img/self_attention_calc_partial.png)

<!-- <img src="img/self_attention_calc_partial.png" width="300" height="300"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

### Transformer blocks

- In many advanced architectures, you will see transformer blocks which consists of
    - The self-attention layer
    - Additional feedforward layers
    - Residual connections
    - Normalizing layers

![](img/transformer_block.png)

<!-- <img src="img/transformer_block.png" width="350" height="350"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

- The input and output dimensions of these layers are matched so that they can be stacked. 
- In deep networks, **residual connections** are connections that pass information from a lower layer to a higher layer without going through the intermediate layer. Why? It has been shown that allowing information from the activation going forward and the gradient going backwards to skip a layer improves learning and gives higher level layers direct access to information from lower layers. 
- We then have a summed vector (projected output of the attention or feedforward layer + input of the attention or feedforward layers). 
- **Layer normalization or layer norm** normalizes the resulting vector which improves training performance in deep neural networks keeping the values of a hidden layer in a range that facilitates gradient-based training. Layer norm applies something similar to `StandardScaler` so that the mean is 0 and standard deviation is 1 in the vector. 

<br><br><br><br>

<br><br><br><br>

## ❓❓ Questions for you

iClicker cloud join link: https://join.iclicker.com/4QVT4

### Exercise 7.1: Select all of the following statements which are **True** (iClicker)

- (A) The main difference between the RNN layer and a self-attention layer is that 
in self-attention, we pass the information without intermediate recurrent connections. 
- (B) In self-attention, the output $y_i$ of input $x_i$ at time $i$ is a single number. 
- (C) Calculating attention weights is quadratic in the length of the input 
since we need to compute dot products between each pair of tokens in
the input.  
- (D) Self-attention results in contextual embeddings. 
- (E) Transformers seem to be more intuitive compared to LSTMs. 

<br><br><br><br>

```{admonition} Exercise 8.2: V's Solutions!
:class: tip, dropdown
- (A) True
- (B) False
- (C) True
- (D) True
- (E) True (for me)
```

<br><br><br><br>

## Multi-head attention

- Different words in a sentence can relate to each other in many different ways simultaneously. 
- Consider the sentence below. 
> The cat was scared because it didn't recognize me in my mask. 

Let's look at all the dependencies in this sentence. 

In [2]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_md")

In [3]:
doc = nlp("I gave my cat some food")
displacy.render(doc, style="dep")

- So a single attention layer usually is not enough to capture all different kinds of parallel relations between inputs. 
- Transformers address this issue with **multihead self-attention layers**.
- These self-attention layers are called **heads**.
- They are at the same depth of the model, operate in parallel, each with a different set of parameters. 
- The idea is that with these different sets of parameters, each head can learn different aspects of the relationships that exist among inputs.

\begin{equation}
\begin{split}
MultiHeadAttn(X) &= (\text{head}_1 \oplus \text{head}_2 \dots \oplus \text{head}_h)W^O\\
               Q &= XW_i^Q ; K = XW_i^K ; V = XW_i^V\\
               \text{head}_i &= SelfAttention(Q,K,V)
\end{split}
\end{equation}

![](img/multihead_attention.png)

<!-- <img src="img/multihead_attention.png" width="600" height="600"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

### Multi-head attention visualization
- Similar to RNNs you can stack self-attention layers or multihead self-attention layers on the top of each other.
- Let's look at this visualization which shows where the attention of different attention heads is going in multihead attention. 
    - [Multi-head attention interactive visualization](https://colab.research.google.com/github/tensorflow/tensor2tensor/blob/master/tensor2tensor/notebooks/hello_t2t.ipynb#scrollTo=OJKU36QAfqOC)

### Let's try it out with PyTorch

In [4]:
from torch.nn import MultiheadAttention

In [5]:
embed_dim = 12
num_heads = 4
seqlen = 20

multi_attn = nn.MultiheadAttention(embed_dim, num_heads)

In [6]:
query = torch.rand(10, embed_dim)  # target_seq_len, query_embedding_dim
key = torch.rand(seqlen, embed_dim)  # source_seq_len, key_embedding_dim
value = torch.rand(seqlen, embed_dim)  # source_seq_len, value_embedding_dim

In [7]:
attn_output, attn_output_weights = multi_attn(query, key, value)

In [8]:
attn_output.shape

torch.Size([10, 12])

In [9]:
attn_output

tensor([[-1.0771e-01, -4.3447e-03, -4.0930e-02,  6.0612e-01,  5.6192e-01,
         -1.5399e-02, -3.7981e-01, -1.6622e-01,  5.1085e-02,  5.7880e-01,
         -1.5687e-03,  1.9999e-01],
        [-1.0806e-01, -2.4403e-03, -4.2450e-02,  6.0569e-01,  5.6444e-01,
         -1.5939e-02, -3.8356e-01, -1.6585e-01,  5.1074e-02,  5.7944e-01,
          1.4274e-03,  2.0143e-01],
        [-1.0800e-01, -2.5962e-03, -4.1494e-02,  6.0620e-01,  5.6307e-01,
         -1.5273e-02, -3.8123e-01, -1.6550e-01,  5.0124e-02,  5.7931e-01,
         -3.5028e-04,  2.0030e-01],
        [-1.0712e-01, -2.3822e-03, -4.1292e-02,  6.0639e-01,  5.6253e-01,
         -1.5737e-02, -3.7916e-01, -1.6535e-01,  4.9301e-02,  5.7799e-01,
         -1.4083e-03,  1.9952e-01],
        [-1.0907e-01, -1.8094e-03, -4.1916e-02,  6.0795e-01,  5.6262e-01,
         -1.4327e-02, -3.8275e-01, -1.6536e-01,  4.9692e-02,  5.8252e-01,
          2.2621e-04,  1.9918e-01],
        [-1.0959e-01, -2.1545e-03, -4.1269e-02,  6.0777e-01,  5.6223e-01,
      

In [10]:
attn_output_weights.shape

torch.Size([10, 20])

In [11]:
attn_output_weights.shape

torch.Size([10, 20])

<br><br>

<br><br><br><br>

## Resources
Attention-mechanisms and transformers are quite new. But there are many resources on transformers. I'm listing a few resources here. 

- [Attention is all you need](https://arxiv.org/pdf/1706.03762.pdf)
- [Transformers](https://ai.googleblog.com/2017/08/transformer-novel-neural-network.html)
- [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
- [Transformers documentation](https://huggingface.co/transformers/index.html)
- [A funny video: I taught an AI to make pasta](https://www.youtube.com/watch?v=Y_NvR5dIaOY)
- [BERT](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)

<br><br><br><br>